In [1]:
import sagemaker
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import boto3

In [2]:
session = sagemaker.Session()

prefix = 'capstone_starbucks_project'

data_dir = './data'

# create an S3 bucket
bucket = session.default_bucket()
output_path='s3://{}/{}/'.format(bucket, prefix)

In [13]:
bucket

'sagemaker-us-east-2-330335126841'

In [3]:
from sagemaker import get_execution_role

role = get_execution_role()

In [4]:
input_location = session.upload_data(path=data_dir,bucket=bucket, key_prefix=prefix)
print('test location: ',input_location)

test location:  s3://sagemaker-us-east-2-330335126841/capstone_starbucks_project


In [5]:
# directory can be changed to: source_sklearn or source_pytorch
# !pygmentize source/train.py

In [6]:
from sagemaker import LinearLearner

In [7]:
ll = LinearLearner(role=role,train_instance_count=1,train_instance_type='ml.c4.xlarge',output_path=output_path, predictor_type='regressor')

In [8]:
bogo_train = pd.read_csv('./data/bogo_train_data.csv',header=None)
X_train = bogo_train.iloc[:,1:]
y_train = bogo_train.iloc[:,:1]
X_train_np = X_train.values.astype('float32')
y_train_np = y_train.values.astype('float32').squeeze()
formatted_bogo_train = ll.record_set(train=X_train_np,labels=y_train_np)

In [54]:
ll.fit(formatted_bogo_train)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-27 16:07:45 Starting - Starting the training job...
2020-10-27 16:07:47 Starting - Launching requested ML instances.........
2020-10-27 16:09:19 Starting - Preparing the instances for training......
2020-10-27 16:10:40 Downloading - Downloading input data...
2020-10-27 16:11:14 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/27/2020 16:11:19 INFO 140526920525632] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'in

In [16]:
# Get the name of the training job, it's suggested that you copy-paste
# from the notebook or from a specific job in the AWS console
training_job_name='linear-learner-2020-10-27-16-07-45-684'

# where the model is saved, by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# download and unzip model
boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

capstone_starbucks_project/linear-learner-2020-10-27-16-07-45-684/output/model.tar.gz


256

In [17]:
import mxnet as mx

# loading the unzipped artifacts
ll_model_params = mx.ndarray.load('model_algo-1')

# what are the params
print(ll_model_params)

MXNetError: [16:40:32] src/ndarray/ndarray.cc:1849: Check failed: header == kMXAPINDArrayListMagic: Invalid NDArray file format
Stack trace:
  [bt] (0) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x32952b) [0x7f715a68852b]
  [bt] (1) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(mxnet::NDArray::Load(dmlc::Stream*, std::vector<mxnet::NDArray, std::allocator<mxnet::NDArray> >*, std::vector<std::string, std::allocator<std::string> >*)+0x161) [0x7f715da71b21]
  [bt] (2) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(MXNDArrayLoad+0x23c) [0x7f715d7d93ac]
  [bt] (3) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7f71b8a16ec0]
  [bt] (4) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7f71b8a1687d]
  [bt] (5) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/pytho

In [ ]:
ll.

In [80]:
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

output_path = 's3://{}/{}'.format(bucket,prefix)

estimator = SKLearn(entry_point='train.py',
                    source_dir='source',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    sagemaker_session=session,
                    output_path=output_path)

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [81]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train':train_location})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-27 12:35:31 Starting - Starting the training job...
2020-10-27 12:35:34 Starting - Launching requested ML instances......
2020-10-27 12:36:55 Starting - Preparing the instances for training......
2020-10-27 12:37:43 Downloading - Downloading input data...
2020-10-27 12:38:27 Training - Training image download completed. Training in progress.
2020-10-27 12:38:27 Uploading - Uploading generated training model.2020-10-27 12:38:22,788 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-10-27 12:38:22,791 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-27 12:38:22,803 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-10-27 12:38:23,067 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-10-27 12:38:23,068 sagemaker-containers INFO     Generating setup.cfg
2020-10-27 12:38:23,068 sagemaker-containers INFO     Generating MANIFEST.in


UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2020-10-27-12-35-31-612: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/miniconda3/bin/python -m train"

In [24]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(session.boto_region_name, 'LinearLearner')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


ValueError: Algorithm class:LinearLearner does not have mapping to account_id with images